In [22]:
import pandas as pd
import numpy as np
from yahoo_fin.stock_info import get_data
import statsmodels.formula.api as smf
from pypfopt.efficient_frontier import EfficientFrontier
import datetime
import requests 
from bs4 import BeautifulSoup
from scrapy import Selector
import json
import time

In [23]:
START = TRAIN_START = datetime.datetime(2017,1,1)
TEST_START = datetime.datetime(2020,1,5)
END = datetime.datetime(2020,12,31)

In [24]:
CC100_Index = pd.read_csv("indexReturns-BITW100_temp.csv")
CC100_Index['date'] = pd.to_datetime(CC100_Index['date']).dt.tz_localize(None)
CC100_Index = CC100_Index.sort_values('date').set_index('date')
CC100_Index = CC100_Index.truncate(before = START,after = END)
# CC100_Index = CC100_Index.resample('W').last()[:-1]
CC100_Return = np.log(CC100_Index/ CC100_Index.shift(1)).fillna(axis = 0, method ='bfill').rename(columns = {"MarketIndex":"MarketReturn"})
CC100 = pd.concat([CC100_Index, CC100_Return],axis =1)

In [25]:
CC100

,MarketIndex,MarketReturn
date,,
2017-01-01 00:00:00,964.00,0.032656
2017-01-02 00:00:00,996.00,0.032656
2017-01-03 00:00:00,1019.82,0.023634
2017-01-04 00:00:00,1047.76,0.027028
2017-01-05 00:00:00,1163.81,0.105045
...,...,...
2020-12-26 21:00:00,24145.53,0.069255
2020-12-27 21:00:00,23984.01,-0.006712
2020-12-28 21:00:00,24895.42,0.037296


In [26]:
CC_string = "ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX"
CC_temp_list = CC_string.split(",")
CC_list = [i+'-USD' for i in CC_temp_list]

# Fetch and save data of all 72 CC from yahoo finance

In [27]:
# data_all = pd.DataFrame()
# for i in CC_list:
#     print(i)
#     data = get_data(i,start_date = START,end_date = END).resample('W').last()[:-1]
#     data_all = pd.concat([data_all,data])
# data_all.index.names = ['date']
# data_all.to_csv('CC_weekly_data.csv')

In [28]:
# data_all = pd.DataFrame()
# for i in CC_list:
#     print(i)
#     data = get_data(i,start_date = START,end_date = END)[:-1]
#     data_all = pd.concat([data_all,data])
# data_all.index.names = ['date']
# data_all.to_csv('CC_daily_data.csv')

# Read data of all 72 CC 

In [29]:
data_copy = pd.read_csv('CC_daily_data.csv')
data_copy['ticker'] = data_copy['ticker'].str.replace('-USD',' ')
data_copy = data_copy.copy().reset_index().set_index(['ticker','date'])

In [30]:
data_copy

index      open      high       low     close  adjclose  \
ticker date                                                                  
ADA    2017-09-30      0  0.021678  0.032226  0.017354  0.024969  0.024969   
       2017-10-01      1  0.024607  0.030088  0.019969  0.025932  0.025932   
       2017-10-02      2  0.025757  0.027425  0.020690  0.020816  0.020816   
       2017-10-03      3  0.020864  0.022806  0.020864  0.021931  0.021931   
       2017-10-04      4  0.021951  0.022154  0.020859  0.021489  0.021489   
...                  ...       ...       ...       ...       ...       ...   
ZRX    2020-12-26  97396  0.358214  0.366815  0.350426  0.354228  0.354228   
       2020-12-27  97397  0.354252  0.424531  0.354252  0.373358  0.373358   
       2020-12-28  97398  0.373753  0.394154  0.370035  0.376743  0.376743   
       2020-12-29  97399  0.376773  0.383101  0.349014  0.365462  0.365462   
       2020-12-30  97400  0.365471  0.369798  0.349509  0.359764  0.359764   

                        volume  
ticker date                     
ADA    2017-09-30   50068700.0  
       2017-10-01   57641300.0  
       2017-10-02   16997800.0  
       2017-10-03    9000050.0  
       2017-10-04    5562510.0  
...                        ...  
ZRX    2020-12-26   43885187.0  
       2020-12-27  142214520.0  
       2020-12-28   59466214.0  
       2020-12-29   54115513.0  
       2020-12-30   39904606.0  

[97401 rows x 7 columns]

# Save top 10 liquid CC

In [31]:
data_copy_copy = data_copy.copy().reset_index().set_index(['date', 'ticker'])
top10_liquid = pd.DataFrame(columns = ['date', 'top10_Liquid'])

for date in CC100.index:
    date = str(date).split(' ')[0]
    if not (date in list(data_copy_copy.reset_index()['date'])):
        print(date)
        continue
    CC_list = data_copy_copy.loc[date].sort_values('volume',ascending= False).iloc[:10].index
    Append = {'date': date, 'top10_Liquid' : CC_list.tolist()}
    top10_liquid = top10_liquid.append(Append, ignore_index=True)

In [32]:
top10_liquid

,date,top10_Liquid
0,2017-01-01,"[BTC , ETH , LTC , XMR , DASH , ETC , ZEC , MA..."
1,2017-01-02,"[BTC , LTC , ETH , XMR , DASH , ETC , XRP , MA..."
2,2017-01-03,"[BTC , ETH , XMR , LTC , ETC , DASH , ZEC , XR..."
3,2017-01-04,"[BTC , ETH , LTC , XMR , ETC , DASH , ZEC , XR..."
4,2017-01-05,"[BTC , ETH , LTC , XMR , ETC , XRP , ZEC , DAS..."
...,...,...
1452,2020-12-26,"[BTC , ETH , LTC , XRP , BCH , EOS , LINK , XM..."
1453,2020-12-27,"[BTC , ETH , LTC , XRP , BCH , EOS , LINK , AD..."
1454,2020-12-28,"[BTC , ETH , LTC , XRP , BCH , EOS , ADA , LIN..."
1455,2020-12-29,"[BTC , ETH , XRP , LTC , BCH , ADA , EOS , LIN..."


# Save top 10 market cap CC

In [33]:
def top10_Market(date):
    date = date.replace('-', '')
    url = 'https://coinmarketcap.com/historical/' + date
    cmc = requests.get(url)
    soup = BeautifulSoup(cmc.content, 'html.parser')
    data = soup.find('script', id='__NEXT_DATA__',type='application/json')
    coin_data = json.loads(data.contents[0])
    listings = coin_data['props']['initialState']['cryptocurrency']['listingHistorical']['data']
    df = pd.DataFrame(listings)
    df = df[['symbol']]
    CC_string = "ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX"
    CC_temp_list = CC_string.split(",")
    df = df[df['symbol'].isin(CC_temp_list)] 
    return df['symbol']

In [34]:
price_top10_Market = pd.DataFrame(columns = ['Date', 'top10_Market'])

i = 1

for date in CC100.index:
    print(date)
    date = str(date).split(' ')[0]
#     print(top10_Market(date))
    CC_list_temp = top10_Market(date).iloc[:10]
    Append = {'Date': date, 'top10_Market' : CC_list_temp.tolist()}
    price_top10_Market = price_top10_Market.append(Append, ignore_index=True)

    i = i+1
    if i == 5:
        i = 0
        time.sleep(120)


2017-01-01 00:00:00
2017-01-02 00:00:00
2017-01-03 00:00:00
2017-01-04 00:00:00
2017-01-05 00:00:00
2017-01-06 00:00:00
2017-01-07 00:00:00
2017-01-08 00:00:00
2017-01-09 00:00:00
2017-01-10 00:00:00
2017-01-11 00:00:00
2017-01-12 00:00:00
2017-01-13 00:00:00
2017-01-14 00:00:00
2017-01-15 00:00:00
2017-01-16 00:00:00
2017-01-17 00:00:00
2017-01-18 00:00:00
2017-01-19 00:00:00
2017-01-20 00:00:00
2017-01-21 00:00:00
2017-01-22 00:00:00
2017-01-23 00:00:00
2017-01-24 00:00:00
2017-01-25 00:00:00
2017-01-26 00:00:00
2017-01-27 00:00:00
2017-01-28 00:00:00
2017-01-29 00:00:00
2017-01-30 00:00:00
2017-01-31 00:00:00
2017-02-01 00:00:00
2017-02-02 00:00:00
2017-02-03 00:00:00
2017-02-04 00:00:00
2017-02-05 00:00:00
2017-02-06 00:00:00
2017-02-07 00:00:00
2017-02-08 00:00:00
2017-02-09 00:00:00
2017-02-10 00:00:00
2017-02-11 00:00:00
2017-02-12 00:00:00
2017-02-13 00:00:00
2017-02-14 00:00:00
2017-02-15 00:00:00
2017-02-16 00:00:00
2017-02-17 00:00:00
2017-02-18 00:00:00
2017-02-19 00:00:00


KeyError: "None of [Index(['symbol'], dtype='object')] are in the [columns]"

In [35]:
price_top10_Market

,Date,top10_Market
0,2017-01-01,"[BTC, ETH, XRP, LTC, XMR, ETC, DASH, REP, MAID..."
1,2017-01-02,"[BTC, ETH, XRP, LTC, XMR, ETC, DASH, REP, MAID..."
2,2017-01-03,"[BTC, ETH, XRP, LTC, XMR, ETC, DASH, REP, MAID..."
3,2017-01-04,"[BTC, ETH, XMR, XRP, LTC, ETC, DASH, REP, MAID..."
4,2017-01-05,"[BTC, ETH, XRP, XMR, LTC, ETC, DASH, REP, MAID..."
...,...,...
324,2017-11-22,"[BTC, ETH, BCH, XRP, DASH, LTC, XMR, MIOTA, NE..."
325,2017-11-23,"[BTC, ETH, BCH, XRP, DASH, LTC, XMR, NEO, MIOT..."
326,2017-11-24,"[BTC, ETH, BCH, XRP, DASH, LTC, XMR, NEO, MIOT..."
327,2017-11-25,"[BTC, ETH, BCH, XRP, BTG, DASH, LTC, XMR, NEO,..."


In [36]:
top10CC = pd.concat([top10_liquid, price_top10_Market], axis=1)
top10CC = top10CC.drop('Date', axis = 1)
top10CC = top10CC.set_index('date')
top10CC

,top10_Liquid,top10_Market
date,,
2017-01-01,"[BTC , ETH , LTC , XMR , DASH , ETC , ZEC , MA...","[BTC, ETH, XRP, LTC, XMR, ETC, DASH, REP, MAID..."
2017-01-02,"[BTC , LTC , ETH , XMR , DASH , ETC , XRP , MA...","[BTC, ETH, XRP, LTC, XMR, ETC, DASH, REP, MAID..."
2017-01-03,"[BTC , ETH , XMR , LTC , ETC , DASH , ZEC , XR...","[BTC, ETH, XRP, LTC, XMR, ETC, DASH, REP, MAID..."
2017-01-04,"[BTC , ETH , LTC , XMR , ETC , DASH , ZEC , XR...","[BTC, ETH, XMR, XRP, LTC, ETC, DASH, REP, MAID..."
2017-01-05,"[BTC , ETH , LTC , XMR , ETC , XRP , ZEC , DAS...","[BTC, ETH, XRP, XMR, LTC, ETC, DASH, REP, MAID..."
...,...,...
2020-12-26,"[BTC , ETH , LTC , XRP , BCH , EOS , LINK , XM...",NaN
2020-12-27,"[BTC , ETH , LTC , XRP , BCH , EOS , LINK , AD...",NaN
2020-12-28,"[BTC , ETH , LTC , XRP , BCH , EOS , ADA , LIN...",NaN


In [ ]:
# top10CC['top10_Liquid'] = top10CC['top10_Liquid'].str.replace('[','')
# top10CC['top10_Liquid'] = top10CC['top10_Liquid'].str.replace(']','')
# top10CC['top10_Liquid'] = top10CC['top10_Liquid'].str.replace(' \'','')
# top10CC['top10_Liquid'] = top10CC['top10_Liquid'].str.replace('\'','')

# top10CC['top10_Market'] = str(top10CC['top10_Market']).replace('[','')
# top10CC['top10_Market'] = str(top10CC['top10_Market']).replace(']','')
# top10CC['top10_Market'] = str(top10CC['top10_Market']).replace(' \'','')
# top10CC['top10_Market'] = str(top10CC['top10_Market']).replace('\'','')

# top10CC

In [37]:
top10CC.to_csv('top10CC_daily.csv')